# Datenvorbereitung
### Abstracts einlesen

In [7]:
with open("../PaperAbstracts_CS_nonPatent.txt") as f:
    abstracts = f.readlines()
f.close()
# Remove whitespaces
abstracts = [x.split("\t", 1)[1].strip() for x in abstracts]
print(abstracts[0:5])

['The purpose of this study is to develop a learning tool for high school students studying the scientific aspects of information and communication networks. More specifically, we focus on the basic principles of network protocols as the aim to develop our learning tool. Our tool gives students hands-on experience to help understand the basic principles of network protocols.', "This paper describes the design and implementation of a methodology for the visualisation and hypothetical virtual reconstruction of Roman polychrome statuary for research purposes. The methodology is intended as an attempt to move beyond visualisations which are simply believable towards a more physically accurate approach. Accurate representations of polychrome statuary have great potential utility both as a means of illustrating existing interpretations and as a means of testing and revising developing hypotheses. The goal of this methodology is to propose a pipeline which incorporates a high degree of physic

In [8]:
len(abstracts)

7365733

### In Sätze aufteilen

In [9]:
import nltk
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [10]:
with open('data/full_text_corpus.txt', 'w', encoding='utf-8') as f:
    for abstract in abstracts:
        lines = (sent_detector.tokenize(abstract.strip()))
        if len(lines) < 3:
            continue

        # This will be used for the training of word2vec and doc2vec
        # You need to create the data folder beforehand
        for line in lines:
            if len(line) < 10:
                continue
            f.write(line + "\n")

### Doc2Vec und Word2Vec

In [11]:
import gensim
from gensim.models import Word2Vec, Doc2Vec, Phrases
from gensim.models.phrases import Phraser
import multiprocessing

with open('data/full_text_corpus.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
len(lines)

40139652

Word2Vec

In [2]:
with open('data/tud_data2vec.txt', 'r') as fin:
    lines = [line.lower() for line in fin]
    print("Lines loaded")
    with open('data/tud_data2vec.txt', 'w') as out:
        out.writelines(lines)
        
#%run data_preparation/train_word2vec.py data/tud_data2vec.txt embedding_models/modelword2vecbigram.model embedding_models/modelword2vecbigram.vec
"""
sentence_stream = [[w for w in nltk.word_tokenize(line)] for line in lines]
bigram_phrases = Phrases(sentence_stream, min_count=2, threshold=2)
bigram_transformer = Phraser(bigram_phrases)
tokens_ = bigram_transformer[sentence_stream]
model = Word2Vec(tokens_, size=100, window=5, min_count=2, workers=multiprocessing.cpu_count(), sg=1)

model.save("embedding_models/modelword2vecbigram.model")
model.wv.save_word2vec_format("embedding_models/modelword2vecbigram.vec", binary=False)
"""

Lines loaded


'\nsentence_stream = [[w for w in nltk.word_tokenize(line)] for line in lines]\nbigram_phrases = Phrases(sentence_stream, min_count=2, threshold=2)\nbigram_transformer = Phraser(bigram_phrases)\ntokens_ = bigram_transformer[sentence_stream]\nmodel = Word2Vec(tokens_, size=100, window=5, min_count=2, workers=multiprocessing.cpu_count(), sg=1)\n\nmodel.save("embedding_models/modelword2vecbigram.model")\nmodel.wv.save_word2vec_format("embedding_models/modelword2vecbigram.vec", binary=False)\n'

Doc2Vec

In [13]:
TaggedDocument = gensim.models.doc2vec.TaggedDocument
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc.split(), tags=[self.labels_list[idx]])

In [14]:
%run data_preparation/train_doc2vec.py data/tud_data2vec.txt embedding_models/doc2vec.model
#labeled_sentences = LabeledLineSentence(lines, range(len(lines)))
#model = Doc2Vec(vector_size=100, window=10, min_count=5, workers=multiprocessing.cpu_count(),
#                epochs=5, alpha=0.1, min_alpha=0.025)  # use decaying learning rate

#model.build_vocab(labeled_sentences)
#model.train(labeled_sentences, total_examples=model.corpus_count, epochs=model.epochs)
#model.save("embedding_models/doc2vec.model")

2020-05-28 22:11:49,718: INFO: running data_preparation/train_doc2vec.py data/tud_data2vec.txt embedding_models/doc2vec.model
2020-05-28 22:11:55,902: INFO: collecting all words and their counts
/home/kd-sem-ie/SmartPub-TSENER/data_preparation/train_doc2vec.py:45: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  yield LabeledSentence(words=doc.split(), tags=[self.labels_list[idx]])
2020-05-28 22:11:55,903: INFO: PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags


55261
55261


2020-05-28 22:11:56,126: INFO: PROGRESS: at example #10000, processed 248343 words (1118674/s), 31425 word types, 10000 tags
2020-05-28 22:11:56,348: INFO: PROGRESS: at example #20000, processed 489325 words (1090232/s), 50053 word types, 20000 tags
2020-05-28 22:11:56,569: INFO: PROGRESS: at example #30000, processed 729132 words (1088800/s), 65852 word types, 30000 tags
2020-05-28 22:11:56,795: INFO: PROGRESS: at example #40000, processed 972797 words (1081824/s), 80487 word types, 40000 tags
2020-05-28 22:11:57,019: INFO: PROGRESS: at example #50000, processed 1212462 words (1076407/s), 93630 word types, 50000 tags
2020-05-28 22:11:57,136: INFO: collected 101310 word types and 55261 unique tags from a corpus of 55261 examples and 1340138 words
2020-05-28 22:11:57,137: INFO: Loading a fresh vocabulary
2020-05-28 22:11:57,830: INFO: effective_min_count=5 retains 15389 unique words (15% of original 101310, drops 85921)
2020-05-28 22:11:57,831: INFO: effective_min_count=5 leaves 1223724

2020-05-28 22:13:11,646: INFO: EPOCH 4 - PROGRESS: at 79.46% examples, 71483 words/s, in_qsize 9, out_qsize 0
2020-05-28 22:13:12,657: INFO: EPOCH 4 - PROGRESS: at 87.76% examples, 72150 words/s, in_qsize 9, out_qsize 0
2020-05-28 22:13:13,757: INFO: EPOCH 4 - PROGRESS: at 96.92% examples, 72768 words/s, in_qsize 5, out_qsize 0
2020-05-28 22:13:13,767: INFO: worker thread finished; awaiting finish of 4 more threads
2020-05-28 22:13:13,818: INFO: worker thread finished; awaiting finish of 3 more threads
2020-05-28 22:13:13,826: INFO: worker thread finished; awaiting finish of 2 more threads
2020-05-28 22:13:13,837: INFO: worker thread finished; awaiting finish of 1 more threads
2020-05-28 22:13:13,858: INFO: worker thread finished; awaiting finish of 0 more threads
2020-05-28 22:13:13,859: INFO: EPOCH - 4 : training on 1340138 raw words (977840 effective words) took 13.1s, 74521 effective words/s
2020-05-28 22:13:14,879: INFO: EPOCH 5 - PROGRESS: at 5.91% examples, 58116 words/s, in_qsi

### Elasticsearch

In [18]:
import elasticsearch
from elasticsearch import helpers

es = elasticsearch.Elasticsearch([{'host': 'localhost', 'port': 9200}],
                                 timeout=30, max_retries=10, retry_on_timeout=True)

In [19]:
es.cluster.health()

{'cluster_name': 'docker-cluster',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 7,
 'active_shards': 7,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 77.77777777777779}

In [ ]:
actions = []
with open("../PaperAbstracts_CS_nonPatent.txt") as f:
    for abstract in f.readlines():
        split = abstract.split("\t", 1)
        actions.append({
            "_index": "ir", 
            "_type": "publications",  
            "_id": split[0],
            "_source": {
                "abstract": split[1].strip()
            }
        })
res = helpers.bulk(es, actions)

/usr/local/lib/python3.8/site-packages/elasticsearch/helpers/actions.py:122: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  resp = client.bulk("\n".join(bulk_actions) + "\n", *args, **kwargs)
2020-05-29 08:00:27,589: INFO: POST http://localhost:9200/_bulk [status:200 request:0.639s]
2020-05-29 08:00:27,709: INFO: POST http://localhost:9200/_bulk [status:200 request:0.085s]
2020-05-29 08:00:27,832: INFO: POST http://localhost:9200/_bulk [status:200 request:0.093s]
2020-05-29 08:00:27,962: INFO: POST http://localhost:9200/_bulk [status:200 request:0.101s]
2020-05-29 08:00:28,093: INFO: POST http://localhost:9200/_bulk [status:200 request:0.103s]
2020-05-29 08:00:28,204: INFO: POST http://localhost:9200/_bulk [status:200 request:0.082s]
2020-05-29 08:00:28,314: INFO: POST http://localhost:9200/_bulk [status:200 request:0.082s]
2020-05-29 08:00:28,433: INFO: POST http://localhost:9200/_bulk [status:200 request:0.089s]
2020-05-29 08:00:28

2020-05-29 08:00:38,896: INFO: POST http://localhost:9200/_bulk [status:200 request:0.090s]
2020-05-29 08:00:39,012: INFO: POST http://localhost:9200/_bulk [status:200 request:0.087s]
2020-05-29 08:00:39,142: INFO: POST http://localhost:9200/_bulk [status:200 request:0.093s]
2020-05-29 08:00:39,256: INFO: POST http://localhost:9200/_bulk [status:200 request:0.086s]
2020-05-29 08:00:39,442: INFO: POST http://localhost:9200/_bulk [status:200 request:0.158s]
2020-05-29 08:00:39,614: INFO: POST http://localhost:9200/_bulk [status:200 request:0.134s]
2020-05-29 08:00:39,741: INFO: POST http://localhost:9200/_bulk [status:200 request:0.098s]
2020-05-29 08:00:39,854: INFO: POST http://localhost:9200/_bulk [status:200 request:0.085s]
2020-05-29 08:00:39,969: INFO: POST http://localhost:9200/_bulk [status:200 request:0.087s]
2020-05-29 08:00:40,093: INFO: POST http://localhost:9200/_bulk [status:200 request:0.095s]
2020-05-29 08:00:40,208: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:00:50,273: INFO: POST http://localhost:9200/_bulk [status:200 request:0.088s]
2020-05-29 08:00:50,390: INFO: POST http://localhost:9200/_bulk [status:200 request:0.088s]
2020-05-29 08:00:50,514: INFO: POST http://localhost:9200/_bulk [status:200 request:0.095s]
2020-05-29 08:00:50,649: INFO: POST http://localhost:9200/_bulk [status:200 request:0.107s]
2020-05-29 08:00:50,796: INFO: POST http://localhost:9200/_bulk [status:200 request:0.108s]
2020-05-29 08:00:50,911: INFO: POST http://localhost:9200/_bulk [status:200 request:0.087s]
2020-05-29 08:00:51,027: INFO: POST http://localhost:9200/_bulk [status:200 request:0.086s]
2020-05-29 08:00:51,143: INFO: POST http://localhost:9200/_bulk [status:200 request:0.089s]
2020-05-29 08:00:51,258: INFO: POST http://localhost:9200/_bulk [status:200 request:0.086s]
2020-05-29 08:00:51,374: INFO: POST http://localhost:9200/_bulk [status:200 request:0.088s]
2020-05-29 08:00:51,494: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:01:01,433: INFO: POST http://localhost:9200/_bulk [status:200 request:0.098s]
2020-05-29 08:01:01,572: INFO: POST http://localhost:9200/_bulk [status:200 request:0.106s]
2020-05-29 08:01:01,710: INFO: POST http://localhost:9200/_bulk [status:200 request:0.104s]
2020-05-29 08:01:01,849: INFO: POST http://localhost:9200/_bulk [status:200 request:0.107s]
2020-05-29 08:01:01,985: INFO: POST http://localhost:9200/_bulk [status:200 request:0.102s]
2020-05-29 08:01:02,135: INFO: POST http://localhost:9200/_bulk [status:200 request:0.117s]
2020-05-29 08:01:02,273: INFO: POST http://localhost:9200/_bulk [status:200 request:0.105s]
2020-05-29 08:01:02,414: INFO: POST http://localhost:9200/_bulk [status:200 request:0.107s]
2020-05-29 08:01:02,535: INFO: POST http://localhost:9200/_bulk [status:200 request:0.093s]
2020-05-29 08:01:02,656: INFO: POST http://localhost:9200/_bulk [status:200 request:0.093s]
2020-05-29 08:01:02,777: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:01:12,929: INFO: POST http://localhost:9200/_bulk [status:200 request:0.102s]
2020-05-29 08:01:13,053: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:01:13,177: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:01:13,301: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:01:13,428: INFO: POST http://localhost:9200/_bulk [status:200 request:0.098s]
2020-05-29 08:01:13,563: INFO: POST http://localhost:9200/_bulk [status:200 request:0.107s]
2020-05-29 08:01:13,693: INFO: POST http://localhost:9200/_bulk [status:200 request:0.099s]
2020-05-29 08:01:13,821: INFO: POST http://localhost:9200/_bulk [status:200 request:0.100s]
2020-05-29 08:01:13,943: INFO: POST http://localhost:9200/_bulk [status:200 request:0.093s]
2020-05-29 08:01:14,066: INFO: POST http://localhost:9200/_bulk [status:200 request:0.095s]
2020-05-29 08:01:14,191: INFO: POST http://localhost:9200/_bulk [status:200 requ

...

2020-05-29 08:04:43,082: INFO: POST http://localhost:9200/_bulk [status:200 request:0.124s]
2020-05-29 08:04:43,222: INFO: POST http://localhost:9200/_bulk [status:200 request:0.112s]
2020-05-29 08:04:43,346: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:04:43,478: INFO: POST http://localhost:9200/_bulk [status:200 request:0.103s]
2020-05-29 08:04:43,613: INFO: POST http://localhost:9200/_bulk [status:200 request:0.107s]
2020-05-29 08:04:43,767: INFO: POST http://localhost:9200/_bulk [status:200 request:0.125s]
2020-05-29 08:04:43,902: INFO: POST http://localhost:9200/_bulk [status:200 request:0.106s]
2020-05-29 08:04:44,025: INFO: POST http://localhost:9200/_bulk [status:200 request:0.095s]
2020-05-29 08:04:44,154: INFO: POST http://localhost:9200/_bulk [status:200 request:0.101s]
2020-05-29 08:04:44,279: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:04:44,409: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:04:55,265: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:04:55,406: INFO: POST http://localhost:9200/_bulk [status:200 request:0.114s]
2020-05-29 08:04:55,539: INFO: POST http://localhost:9200/_bulk [status:200 request:0.103s]
2020-05-29 08:04:55,673: INFO: POST http://localhost:9200/_bulk [status:200 request:0.104s]
2020-05-29 08:04:55,815: INFO: POST http://localhost:9200/_bulk [status:200 request:0.113s]
2020-05-29 08:04:55,953: INFO: POST http://localhost:9200/_bulk [status:200 request:0.109s]
2020-05-29 08:04:56,086: INFO: POST http://localhost:9200/_bulk [status:200 request:0.104s]
2020-05-29 08:04:56,219: INFO: POST http://localhost:9200/_bulk [status:200 request:0.104s]
2020-05-29 08:04:56,343: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:04:56,473: INFO: POST http://localhost:9200/_bulk [status:200 request:0.102s]
2020-05-29 08:04:56,602: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:05:08,565: INFO: POST http://localhost:9200/_bulk [status:200 request:0.118s]
2020-05-29 08:05:08,704: INFO: POST http://localhost:9200/_bulk [status:200 request:0.110s]
2020-05-29 08:05:08,921: INFO: POST http://localhost:9200/_bulk [status:200 request:0.189s]
2020-05-29 08:05:09,091: INFO: POST http://localhost:9200/_bulk [status:200 request:0.135s]
2020-05-29 08:05:09,243: INFO: POST http://localhost:9200/_bulk [status:200 request:0.123s]
2020-05-29 08:05:09,391: INFO: POST http://localhost:9200/_bulk [status:200 request:0.116s]
2020-05-29 08:05:09,539: INFO: POST http://localhost:9200/_bulk [status:200 request:0.120s]
2020-05-29 08:05:09,739: INFO: POST http://localhost:9200/_bulk [status:200 request:0.169s]
2020-05-29 08:05:09,918: INFO: POST http://localhost:9200/_bulk [status:200 request:0.149s]
2020-05-29 08:05:10,082: INFO: POST http://localhost:9200/_bulk [status:200 request:0.132s]
2020-05-29 08:05:10,238: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:05:23,371: INFO: POST http://localhost:9200/_bulk [status:200 request:0.141s]
2020-05-29 08:05:23,543: INFO: POST http://localhost:9200/_bulk [status:200 request:0.142s]
2020-05-29 08:05:23,679: INFO: POST http://localhost:9200/_bulk [status:200 request:0.108s]
2020-05-29 08:05:23,836: INFO: POST http://localhost:9200/_bulk [status:200 request:0.128s]
2020-05-29 08:05:23,982: INFO: POST http://localhost:9200/_bulk [status:200 request:0.116s]
2020-05-29 08:05:24,164: INFO: POST http://localhost:9200/_bulk [status:200 request:0.152s]
2020-05-29 08:05:24,317: INFO: POST http://localhost:9200/_bulk [status:200 request:0.116s]
2020-05-29 08:05:24,450: INFO: POST http://localhost:9200/_bulk [status:200 request:0.104s]
2020-05-29 08:05:24,603: INFO: POST http://localhost:9200/_bulk [status:200 request:0.121s]
2020-05-29 08:05:24,767: INFO: POST http://localhost:9200/_bulk [status:200 request:0.130s]
2020-05-29 08:05:24,927: INFO: POST http://localhost:9200/_bulk [status:200 requ

2020-05-29 08:05:37,146: INFO: POST http://localhost:9200/_bulk [status:200 request:0.115s]
2020-05-29 08:05:37,278: INFO: POST http://localhost:9200/_bulk [status:200 request:0.104s]
2020-05-29 08:05:37,434: INFO: POST http://localhost:9200/_bulk [status:200 request:0.127s]
2020-05-29 08:05:37,555: INFO: POST http://localhost:9200/_bulk [status:200 request:0.093s]
2020-05-29 08:05:37,677: INFO: POST http://localhost:9200/_bulk [status:200 request:0.094s]
2020-05-29 08:05:37,800: INFO: POST http://localhost:9200/_bulk [status:200 request:0.094s]
2020-05-29 08:05:37,954: INFO: POST http://localhost:9200/_bulk [status:200 request:0.127s]
2020-05-29 08:05:38,090: INFO: POST http://localhost:9200/_bulk [status:200 request:0.107s]
2020-05-29 08:05:38,213: INFO: POST http://localhost:9200/_bulk [status:200 request:0.095s]
2020-05-29 08:05:38,337: INFO: POST http://localhost:9200/_bulk [status:200 request:0.096s]
2020-05-29 08:05:38,468: INFO: POST http://localhost:9200/_bulk [status:200 requ

In [18]:
res = es.search(index = "ir", body = {"query": {"match": {"abstract" : "netflix"}}}, size = 10)
print(res)

2020-05-28 22:17:31,556: INFO: POST http://localhost:9200/ir/_search?size=10 [status:200 request:0.010s]


{'took': 0, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 8.147346, 'hits': [{'_index': 'ir', '_type': 'publications', '_id': '2513698560', '_score': 8.147346, '_source': {'abstract': "Identifying emerging influential or popular node/item in future on network is a current interest of the researchers. Most of previous works focus on identifying leaders in time evolving networks on the basis of network structure or node's activity separate way. In this paper, we have proposed a hybrid model which considers both, node's structural centrality and recent activity of nodes together. We consider that the node is active when it is receiving more links in a given recent time window, rather than in the whole past life of the node. Furthermore our model is flexible to implement structural rank such as PageRank and webpage click information as activity of the node. For testing the performance 

### Sentence Indexing

In [ ]:
with open("../PaperAbstracts_CS_nonPatent.txt") as f:
    abstracts = f.readlines()
    actions = []
    cleaned = []
    dataset_sent = []
    other_sent = []
    for abstract in abstracts:
        split = abstract.split("\t", 1)
        lines = (sent_detector.tokenize(split[1].strip()))
        if len(lines) < 3:
            continue

        # This will be used for the training of word2vec and doc2vec
        # You need to create the data folder beforehand 
        for i in range(len(lines)):
            if len(lines[i]) < 10:
                continue
            words = nltk.word_tokenize(lines[i])
            word_lengths = [len(x) for x in words]
            average_word_length = sum(word_lengths) / len(word_lengths)
            if average_word_length < 4:
                continue

            two_sentences = ''
            try:
                two_sentences = lines[i] + ' ' + lines[i - 1]
            except:
                two_sentences = lines[i] + ' ' + lines[i + 1]

            dataset_sent.append(two_sentences)

    for num, added_lines in enumerate(dataset_sent):
        actions.append({
            "_index": "twosent",
            "_type": "twosentnorules",
            "_id": split[0] + str(num),
            "_source": {
                "content.chapter.sentpositive": added_lines,
                "paper_id": split[0]
            }})

    res = helpers.bulk(es, actions)

### Doc2Vec Indexing

In [ ]:
actions = []
with open('data/full_text_corpus.txt', 'r', encoding='utf-8') as f:
    sentences = f.readlines()
    count = 0
    docLabels = []
    for i, sent in f:
        try:
            neighbors = sentences[i + 1]
            neighbor_count = count + 1
        except:
            neighbors = sentences[i - 1]
            neighbor_count = count - 1

        docLabels.append(count)
        actions.append({
            "_index": "devtwosentnew",
            "_type": "devtwosentnorulesnew",
            "_id": count,
            "_source": {
                "content.chapter.sentpositive": sent,
                "content.chapter.sentnegtive": neighbors,
                "neighborcount": neighbor_count
            }})
        count = count + 1

res = helpers.bulk(es, actions)
print(res)

In [2]:
import nltk

In [3]:
file = open('data/full_text_corpus.txt', 'r', encoding='utf-8')
text = file.read()
file.close()
sentences = nltk.tokenize.sent_tokenize(text)
print('Sentences ready')

Sentences ready


In [16]:
from elasticsearch import helpers


In [20]:
count = 0
docLabels = []
actions = []

for i, sent in enumerate(sentences):
    try:
        neighbors = sentences[i + 1]
        neighbor_count = count + 1
    except:
        neighbors = sentences[i - 1]
        neighbor_count = count - 1

    docLabels.append(count)
    actions.append({
        "_index": "devtwosentnew",
        "_type": "devtwosentnorulesnew",
        "_id": count,
        "_source": {
            "content.chapter.sentpositive": sent,
            "content.chapter.sentnegtive": neighbors,
            "neighborcount": neighbor_count
        }})
    count = count + 1

print(len(sentences))
print(len(docLabels))
res = helpers.bulk(es, actions)
print(res)

39804770
39804770


/usr/local/lib/python3.8/site-packages/elasticsearch/helpers/actions.py:122: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  resp = client.bulk("\n".join(bulk_actions) + "\n", *args, **kwargs)


(39804770, [])
